In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("/Users/larsankile/Code/liquidity-distribution-history/")

In [10]:
from pool_state import v3Pool

import pandas as pd
from prisma import Prisma


from dotenv import load_dotenv


load_dotenv(override=True)

True

In [14]:
client = Prisma()
await client.connect()

await client.swap.count()

In [21]:
swaps_v3 = await client.swap.find_many()

In [39]:
df = pd.DataFrame([swap.dict() for swap in swaps_v3])

print(f"{df.block_number.nunique():,} blocks from {df.block_timestamp.min()} to {df.block_timestamp.max()}")

df.head()

56,838 blocks from 2023-06-16 22:24:23+00:00 to 2023-06-25 04:11:59+00:00


,transaction_hash,block_timestamp,block_number,transaction_index,log_index,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick,address,from_address,to_address,from_mempool
0,0x8009a5b544c00d7dbc98f09b44eb34b718db322869b3...,2023-06-16 22:24:23+00:00,17495345,98,295,0xDef1C0ded9bec7F1a1670819833240f027b25EfF,0x3E47965A595054BE56653a1D237A426233309c9E,-8542476354849,455654402709353356,18275694958246292902075973447291,3539836978395738419,108825,0x9e0905249CeEFfFB9605E034b534544684A58BE6,0xDef1C0ded9bec7F1a1670819833240f027b25EfF,0x3E47965A595054BE56653a1D237A426233309c9E,False
1,0x3e2371f3b011c3ca5bb3e21edaf4197cdb609ce0f7c2...,2023-06-16 22:24:35+00:00,17495346,2,27,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xDEF171Fe48CF0115B1d80b88dc8eAB59176FEe57,-1574307151494113403411,1766521636780182160,2641052031392980960130223425,175211772026694486436642,-68027,0xE936f0073549AD8b1fA53583600d629Ba9375161,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xDEF171Fe48CF0115B1d80b88dc8eAB59176FEe57,False
2,0xd0eeb237d2de4c87977dd1d88c64f888e68a4788222f...,2023-06-16 22:24:35+00:00,17495346,119,259,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,38730188992814572393,-607740296228321509,9974327118410483594753610929,83274147314440038309243,-41449,0x058D79a4C6eb5B11d0248993fFA1faa168DDd3C0,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,False
3,0x8280b3f12bd3e6e8530fe4eb6ee805f216644d995fde...,2023-06-16 22:24:47+00:00,17495347,1,5,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,144609078919,-84121696852463761579,1911357499820391692725701436214544,932435692349012659054,201829,0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,False
4,0x8280b3f12bd3e6e8530fe4eb6ee805f216644d995fde...,2023-06-16 22:24:47+00:00,17495347,1,8,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,36152269729,-36184364178,79267273712263644938102111529,117852460482612044,9,0x3416cF6C708Da44DB2624D63ea0AAef7113527C6,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD,False


In [40]:
swapcounts = df.groupby(['block_number', 'address']).count().transaction_hash.sort_values(ascending=False)

In [42]:
print(f'More than 3 swaps: {(swapcounts > 3).sum():,} / {swapcounts.shape[0]:,} ({(swapcounts > 3).sum() / swapcounts.shape[0]:.2%})')
print(f'More than 4 swaps: {(swapcounts > 4).sum():,} / {swapcounts.shape[0]:,} ({(swapcounts > 4).sum() / swapcounts.shape[0]:.2%})')

More than 3 swaps: 4,623 / 281,369 (1.64%)
More than 4 swaps: 2,130 / 281,369 (0.76%)


In [ ]:
# Start with block and pool with the most swaps
block_number = swapcounts.index.get_level_values(0)[0]
pool_address = swapcounts.index.get_level_values(1)[0]

print(f'Block: {block_number}, Pool: {pool_address}')